In [53]:
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm

import scipy.stats as stats

#%matplotlib inline

In [54]:
example_datafile = 'example_data.txt'
filter_config = '../cpp/tests/assembly/connectivity.txt'

fed = open(example_datafile, 'r')
fc = open(filter_config, 'r')

In [55]:
fed = open(example_datafile, 'r')

begin_data = False
dns_data = {}
for line in fed.readlines():
    
    if begin_data:
        
        if "t = " in line:
            tc = float(line[3:])
            dns_data.update({tc:np.empty((0, 3))})
        else:
            data = [s.replace(',', '') for s in line.split()]
            pos = np.array([float(data[i]) for i in range(2, 5)])
            dns_data[tc] = np.vstack([dns_data[tc], pos])
    
    if (line.strip() == "BEGIN DATA"):
        begin_data = True
    
fed.close()

In [56]:
fc = open(filter_config, 'r')

begin_data = False

nodes = {}
conn = {}

for line in fc.readlines():
    
    if begin_data:
    
        data = [s.replace(',', '') for s in line.split()]
        
        if(len(data)>0):
        
            if (data[0]=='N'):
                nid = int(data[1])
                pos = np.array([float(data[i]) for i in range(2, len(data))])
                nodes.update({nid:pos})
                
            elif (data[0]=='E'):
                eid = int(data[2])
                nids = [int(data[i]) for i in range(3, len(data))]
                conn.update({eid:nids})
        
    if (line.strip() == "BEGIN DATA"):
        begin_data = True

print(nodes)
print(conn)
fc.close()

{1: array([0., 0., 0.]), 2: array([1., 0., 0.]), 3: array([1., 1., 0.]), 4: array([0., 1., 0.]), 5: array([0., 0., 1.]), 8: array([1., 0., 1.]), 6: array([1., 1., 1.]), 7: array([0., 1., 1.])}
{10: [1, 2, 3, 4, 5, 8, 6, 7]}


In [57]:
#Assume hex8 for now
def plot_hex(eid, nodes, conn, u, ax):
    #Plot bottom square
    x = np.array([nodes[n]+u[n] for n in conn[eid][:4]] + [nodes[conn[eid][0]]+u[conn[eid][0]]])
    ax.plot(*zip(*x), color='k', marker='o')
    #Plot top square
    x = np.array([nodes[n]+u[n] for n in conn[eid][4:]] + [nodes[conn[eid][4]]+u[conn[eid][4]]])
    ax.plot(*zip(*x), color='k', marker='o')
    #Plot sides
    for i in range(4):
        x = [nodes[conn[eid][i]]+u[conn[eid][i]], nodes[conn[eid][i+4]]+u[conn[eid][i+4]]]
        ax.plot(*zip(*x), color='k', marker='o')

fig = plt.figure(figsize=(9, 9))
ax = fig.add_subplot(111, projection='3d')
u = dict((key, np.zeros(3)) for key in nodes.keys())
plot_hex(10, nodes, conn, u, ax)
plt.show()

In [58]:
us = np.array([[0.296511, -0.100599, 0.288297],\
[0.340768, -0.0339587, 0.321454],\
[0.297521, -0.0599225, 0.386237],\
[0.253264, -0.126563, 0.35308],\
[0.273059, -0.189617, 0.274714],\
[0.317316, -0.122977, 0.307871],\
[0.274069, -0.148941, 0.372654],\
[0.229812, -0.215581, 0.339497]])

u = dict([(key, us[i]) for key, i in zip(nodes.keys(), range(8))])

fig = plt.figure(figsize=(9, 9))
ax = fig.add_subplot(111, projection='3d')
for i, key in enumerate(dns_data.keys()):
    if (i != 1):
        continue
    else:
        print(key)
    data = dns_data[key]
    ax.scatter(*zip(*data), label='t = {0:1.4f}'.format(key))
plot_hex(10, nodes, conn, u, ax)
plt.show()

0.1


In [59]:
us = np.array([[0.602945, -0.204222, 0.561868],
[0.679657, -0.0624096, 0.6374],
[0.592969, -0.126602, 0.760303],
[0.516257, -0.268414, 0.684771],
[0.57752, -0.381413, 0.51808],
[0.654232, -0.239601, 0.593612],
[0.567544, -0.303793, 0.716514],
[0.490832, -0.445605, 0.640982]])



u = dict([(key, us[i]) for key, i in zip(nodes.keys(), range(8))])
print(u)

fig = plt.figure(figsize=(9, 9))
ax = fig.add_subplot(111, projection='3d')
for i, key in enumerate(dns_data.keys()):
    if (i != 2):
        continue
    data = dns_data[key]
    ax.scatter(*zip(*data), label='t = {0:1.4f}'.format(key))
plot_hex(10, nodes, conn, u, ax)

{1: array([ 0.602945, -0.204222,  0.561868]), 2: array([ 0.679657 , -0.0624096,  0.6374   ]), 3: array([ 0.592969, -0.126602,  0.760303]), 4: array([ 0.516257, -0.268414,  0.684771]), 5: array([ 0.57752 , -0.381413,  0.51808 ]), 8: array([ 0.654232, -0.239601,  0.593612]), 6: array([ 0.567544, -0.303793,  0.716514]), 7: array([ 0.490832, -0.445605,  0.640982])}


In [60]:
[1,2,3,]

[1, 2, 3]

In [66]:
#np.random.seed(123)
#pts = np.hstack([micro_us + dns_data[0], dns_data[0]])
#
#isubset = np.random.choice(pts.shape[0], 100, replace=False)
#subset = pts[isubset]
#
#spts = np.array([[s[:3], s[3:]] for s in subset])
#print(spts.shape)
#
#fig = plt.figure(figsize=(9, 9))
#ax = fig.add_subplot(111, projection='3d')
#u = dict((key, np.zeros(3)) for key in nodes.keys())
#plot_hex(10, nodes, conn, u, ax)
#us = np.array([[0.318976, -0.049644, 0.200312],
#[0.420757, -0.0292879, 0.195223],
#[0.441113, -0.07, 0.174866],
#[0.339332, -0.090356, 0.179955],
#[0.313887, -0.07, 0.190134],
#[0.415668, -0.049644, 0.185045],
#[0.436024, -0.090356, 0.164688],
#[0.334243, -0.110712, 0.169777],
#])
#u = dict([(key, us[i]) for key, i in zip(nodes.keys(), range(8))])
#plot_hex(10, nodes, conn, u, ax)
#
#ax.scatter(*zip(*dns_data[0][isubset]), color='b')
#ax.scatter(*zip(*dns_data[.1][isubset]), color='r')
#
#for sp in spts:
#    
#    ax.plot(sp.T[0], sp.T[1], sp.T[2], color='k')
#    
#
#plt.show()


(100, 2, 3)
